In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
os.chdir("/content/gdrive/My Drive/wzk/seg1")
!pip install ttach
# !pip install git+https://github.com/qubvel/segmentation_models.pytorch

## 引入相关库

In [3]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim
from torchsummary import summary
import torch.nn as nn
import torch
from torch.utils.data import Dataset
import torchvision.transforms.functional as tf
from losses import *
from utils import *
from UNet import *
from UNetplusplus import *
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import ttach as tta
import random

In [4]:
# 设置随机数种子
def setup_seed(seed):
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.backends.cudnn.deterministic = True
seed = 1024
setup_seed(seed)

## 读取数据

In [5]:
# 读取数据
train_x_path = './supplementary_modify/dataset1/train/'
train_y_path = './supplementary_modify/dataset1/train_GT/SEG/'
test_x_path = './supplementary_modify/dataset1/test/'
train_x = []
train_y = []
test_x = []
filename_train_x = os.listdir(train_x_path)
train_num = len(filename_train_x)
for i in tqdm(range(train_num)):
    filename = filename_train_x[i]
    img = cv2.imread(train_x_path + filename, -1)
    img = img_standardization(img)
    train_x.append(img)
    filename2 = 'man_seg' + filename[1:]
    img_label = cv2.imread(train_y_path + filename2, -1)
    img_mask = generate_mask(img_label)
    train_y.append(img_mask)
filename_test_x = os.listdir(test_x_path)
filename_test_x.sort()
test_num = len(filename_test_x)
for i in tqdm(range(test_num)):
    filename = filename_test_x[i]
    img_test = cv2.imread(test_x_path + filename, -1)
    img_test = img_standardization(img_test)
    test_x.append(img_test)
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
test_x = np.asarray(test_x)

100%|██████████| 33/33 [00:44<00:00,  1.36s/it]


 ## 数据处理

In [6]:
# 划分训练集和验证集
BATCH_SIZE = 1
sample_num = train_x.shape[0] #样本数量
pic_size = train_x.shape[1]
train_num = int(sample_num*0.85)
val_num = int(sample_num - train_num)

train_data = MyDataSet(train_x, train_y)
test_data = MyDataSet(test_x, type='test')
train, val = random_split(train_data, [train_num, val_num])
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)
print(train_data[0][0].shape)

torch.Size([1, 640, 640])


## 超参数设置

In [7]:
# 超参数设置
lr = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
out_channel = 2
in_channel = 1
deep_supervision=False
model = UNetPlusPlus(in_channel, out_channel, deep_supervision)
# model.apply(init_weights)
model = model.to(device)
summary(model, input_size=(1, 640, 640))
optimizer = optim.Adam(model.parameters(),lr=lr)
# loss = nn.BCEWithLogitsLoss()
# loss = FocalDiceLoss(0.25, 2)
loss = BCEDiceLoss()
# loss 停止下降时改变学习率
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3)
num_epochs = 20


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 640, 640]             640
       BatchNorm2d-2         [-1, 64, 640, 640]             128
              ReLU-3         [-1, 64, 640, 640]               0
            Conv2d-4         [-1, 64, 640, 640]          36,928
       BatchNorm2d-5         [-1, 64, 640, 640]             128
              ReLU-6         [-1, 64, 640, 640]               0
          ConvCell-7         [-1, 64, 640, 640]               0
         MaxPool2d-8         [-1, 64, 320, 320]               0
            Conv2d-9         [-1, 64, 320, 320]          36,864
      BatchNorm2d-10         [-1, 64, 320, 320]             128
             ReLU-11         [-1, 64, 320, 320]               0
           Conv2d-12         [-1, 64, 320, 320]          36,864
      BatchNorm2d-13         [-1, 64, 320, 320]             128
             ReLU-14         [-1, 64, 

## 网络训练

In [ ]:
for epoch in range(20):
    train_loader = tqdm(train_loader)
    train_loss = 0
    model.train()
    for i, (X, Y) in enumerate(train_loader):  # 使用枚举函数遍历train_loader
        X = Variable(X).to(device) #转化数据类型
        #X = Variable(X)
        X = X.float()
        Y = Variable(Y).to(device)
        Y = Y.float()
        outs = model(X)  # 正向传播
        lossvalue = loss(outs, Y[:,0:out_channel,:,:])  # 求损失值
        optimizer.zero_grad()  # 优化器梯度归零
        lossvalue.backward()  # 反向转播，刷新梯度值
        nn.utils.clip_grad_value_(model.parameters(), 1)
        optimizer.step()  # 优化器运行一步
        # 计算损失
        train_loss += float(lossvalue)
    val_loss = 0  # 定义验证损失
    val_js = 0
    max_js = 0
    model.eval() #模型转化为评估模式
    val_loader = tqdm(val_loader)
    for i,(X, Y) in enumerate(val_loader):
        X = Variable(X).to(device)
        X = X.float()
        Y = Variable(Y).to(device)
        with torch.no_grad():
            outs = model(X)
            score = cal_jaccard_binary(outs, Y[:,0:out_channel,:,:])
        lossvalue = 0
        lossvalue += loss(outs, Y[:,0:out_channel,:,:])  # 求损失值
        val_loss += float(lossvalue)
        val_js += score
    scheduler.step(val_js)
    max_js = max(max_js,val_js)
    if max_js == val_js:
      torch.save(model, 'modelunet++.pkl')      
    print("epoch:" + ' ' + str(epoch))
    lr = optimizer.state_dict()['param_groups'][0]['lr']
    print("lr:", lr)
    print("train lose:" + ' ' + str(train_loss / len(train_loader)))
    print("val lose:" + ' ' + str(val_loss / len(val_loader)))
    print("val js:" + ' ' + str(val_js / len(val_loader)))



  0%|          | 0/148 [00:00<?, ?it/s]

epoch: 0
lr: 0.0001
train lose: 0.4494896558289592
val lose: 0.36222392320632935
val js: 0.6913103860486205


  0%|          | 0/148 [00:00<?, ?it/s]

epoch: 1
lr: 0.0001
train lose: 0.2935018983443041
val lose: 0.29992166216726657
val js: 0.7304593250560605


  0%|          | 0/148 [00:00<?, ?it/s]

...

epoch: 17
lr: 1e-05
train lose: 0.10043458197567914
val lose: 0.15954268944484215
val js: 0.7713227617942624


  0%|          | 0/148 [00:00<?, ?it/s]

epoch: 18
lr: 1.0000000000000002e-06
train lose: 0.09813453551583193
val lose: 0.1631071357815354
val js: 0.7667535362797314


100%|██████████| 27/27 [00:05<00:00,  4.96it/s]


epoch: 19
lr: 1.0000000000000002e-06
train lose: 0.0951757037357704
val lose: 0.15289696085232277
val js: 0.7799128163511582


## 验证集输出

In [8]:
model = torch.load('modelunet++.pkl')
postprocess = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()
val_marker,val_border = [],[]
val_pred = []
val_y,val_x = [],[]
for i, (X,Y) in enumerate(val_loader):
    X = Variable(X).to(device)
    X = X.float()
    Y = Variable(Y).to(device)
    with torch.no_grad():
      out = model(X)
      out_prob1 = torch.sigmoid(out[:,0,:,:])
      out_prob2 = torch.sigmoid(out[:,1,:,:])
    y_marker = out_prob1.cpu().clone().numpy()[0,:,:]
    y_border = out_prob2.cpu().clone().numpy()[0,:,:]
    val_marker.append(y_marker)
    val_border.append(y_border)
    x_real = X.cpu().clone().numpy()[0,0,:,:]
    y_real = np.zeros((640,640,3))
    y_real[:,:,0] = Y.cpu().clone().numpy()[0,0,:,:]
    y_real[:,:,1] = Y.cpu().clone().numpy()[0,1,:,:]
    y_real[:,:,2] = Y.cpu().clone().numpy()[0,2,:,:]        
    val_y.append(y_real)
    val_x.append(x_real)
val_marker = np.asarray(val_marker)
val_marker = np.expand_dims(val_marker,3)
val_border = np.asarray(val_border)
val_border = np.expand_dims(val_border,3)
val_x = np.asarray(val_x)
val_x = np.expand_dims(val_x,3)
val_x_new = np.concatenate((val_marker, val_x), axis=3)
val_y = np.asarray(val_y)

## 测试集输出

In [9]:
tta_model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')
test_x = []
test_marker = []
test_border = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for i, X in enumerate(test_loader):
    X = Variable(X).to(device)
    X = X.float()
    with torch.no_grad():
      out = model(X)
      out_prob1 = torch.sigmoid(out[:,0,:,:])
      out_prob2 = torch.sigmoid(out[:,1,:,:])
    y_marker = out_prob1.cpu().clone().numpy()[0,:,:]
    y_border = out_prob2.cpu().clone().numpy()[0,:,:]
    x_real = X.cpu().clone().numpy()[0,0,:,:]
    test_marker.append(y_marker)
    test_border.append(y_border)
    test_x.append(x_real)
test_marker = np.asarray(test_marker)
test_border = np.asarray(test_border)
test_x = np.asarray(test_x)
test_marker = np.expand_dims(test_marker,3)
test_border = np.expand_dims(test_border,3)
test_x = np.expand_dims(test_x,3)
test_x_new = np.concatenate((test_marker, test_x), axis=3)

## 验证集jaccard score

In [10]:
js = 0
s = np.zeros(val_marker.shape[0])
for i in range(val_marker.shape[0]):
  marker = val_marker[i,:,:,0]
  border = val_border[i,:,:,0]
  process = output_postprocess(marker,border)
  labels = (val_y[i,:,:,2]*255).astype(np.uint8)
  s[i] = cal_jaccard(labels, process)
  js += s[i]
js = js/val_marker.shape[0]
print(js)
print(s)

/content/gdrive/My Drive/wzk/seg1/utils.py:184: UserWarning: Any labeled images will be returned as a boolean array. Did you mean to use a boolean array?
  mask2 = remove_small_holes(mask1)


0.7437818875668173
[0.82660545 0.68290434 0.81941004 0.71429614 0.74550364 0.48327137
 0.92028653 0.86009262 0.57198744 0.56274945 0.8142197  0.89772105
 0.81641551 0.78303539 0.7092615  0.72759442 0.87689321 0.74077994
 0.46986786 0.65216621 0.81304077 0.70266726 0.8264449  0.74546959
 0.92150795 0.71985984 0.67805883]


In [13]:
out_path = './test_RES/mask'
for i in range(33):
  marker = test_marker[i,:,:,0]
  border = test_border[i,:,:,0]
  # plt.figure()
  # plt.imshow(marker)
  process = output_postprocess(marker,border)
  process = process[6:634,6:634]
  process = process.astype(np.uint16)
  num = '00' + str(i)
  out_file = out_path + num + '.tif'
  imsave(out_file, process)
  plt.figure()
  plt.imshow(process)
  plt.figure()
  plt.imshow(test_x[i,:,:,0])


Output hidden; open in https://colab.research.google.com to view.